In [0]:
import tensorflow as tf
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [0]:
data = pd.read_csv("clogit_df.csv",encoding='utf-8')

#data = data.dropna() # 이거로 하면 cost 5자리수 나옴
d2 = data
d3=d2.groupby(['distance'])
d3=[d3.get_group(x) for x in d3.groups]
data=d3[0]

y = data['record']
X = data.loc[:, ['lane','age','jockey_w','dandivi','yeondivi',
                 'cure_in_1m','weight_diff', 'raw_weight','weight_added',
                 'prev1_velo','prev2_velo','sex1','sex2']]
#prev3은 도움안됨
'''
# sex는 dummy 함수 처리
sex_mod = pd.get_dummies(X['sex'])
X = X.join(sex_mod)
X = X.drop(columns=['sex'])
#print(X.columns)
'''
X = X.as_matrix()

In [7]:

X

array([[3.4628e+04, 3.0000e+00, 6.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [3.9973e+04, 1.1000e+01, 3.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [3.9806e+04, 6.0000e+00, 3.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       ...,
       [3.0853e+04, 5.0000e+00, 4.0000e+00, ..., 0.0000e+00, 1.0000e+00,
        0.0000e+00],
       [2.9616e+04, 9.0000e+00, 4.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [2.9820e+04, 7.0000e+00, 4.0000e+00, ..., 0.0000e+00, 1.0000e+00,
        0.0000e+00]])

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_mean = []
X_std = []

for i in range(X_train.shape[1]):
  X_mean.append(np.mean(X_train[:,i]))
  X_std.append(np.std(X_train[:,i]))
  print("X_mean:",X_mean[i],"X_std:",X_std[i])
  X_train[:,i] = (X_train[:,i]-X_mean[i])/X_std[i]
  X_test[:,i] = (X_test[:,i]-X_mean[i])/X_std[i]
  
train_size = len(X_train)
print("train_size : ",train_size)

size_x = len(X_train[0])
print("size_x : ",size_x)

# 설정하는 부분
learning_rate = 0.01
training_epochs = 500
batch_size = 128


X_mean: 6.019415309082794 X_std: 3.2803034511394893
X_mean: 3.299709886186119 X_std: 0.8141914694388229
X_mean: 53.58390984155322 X_std: 1.6268055493436016
X_mean: 40.345726400357066 X_std: 57.704866115925995
X_mean: 6.310220932827494 X_std: 7.424858154048369
X_mean: 0.25619281410399464 X_std: 0.4365295592574123
X_mean: 0.12229413077438071 X_std: 7.476497174574059
X_mean: 465.9221155991966 X_std: 26.89509411690668
X_mean: 0.12229413077438071 X_std: 7.476497174574059
X_mean: 15.677243981728417 X_std: 0.4095224128389913
X_mean: 15.646033815443664 X_std: 0.39994614017069147
X_mean: 0.27828609685338096 X_std: 0.44815504588422483
X_mean: 0.5501004240124973 X_std: 0.49748361532191987
train_size :  4481
size_x :  13


In [85]:
print(X_train.shape, X_test.shape)

(4481, 13) (1921, 13)


In [86]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, size_x])
Y = tf.placeholder(tf.float32, [None, 1])
keep_prob = tf.placeholder(tf.float32)
phase = tf.placeholder(tf.bool, name='phase')


W1 = tf.get_variable(shape=[size_x, 128], name='weight1', initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([128]))
layer1 = tf.nn.relu(tf.matmul(X, W1) + b1)
layer1 = tf.nn.dropout(layer1, keep_prob=keep_prob)
layer1 = tf.contrib.layers.batch_norm(layer1, center=True, scale=True, 
                                          is_training=phase)

W2 = tf.get_variable(shape=[128, 64], name='weight2', initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([64]))
layer2 = tf.nn.relu(tf.matmul(layer1, W2) + b2)
layer2 = tf.nn.dropout(layer2, keep_prob=keep_prob)
layer2 = tf.contrib.layers.batch_norm(layer2, center=True, scale=True, 
                                          is_training=phase)

W3 = tf.get_variable(shape=[64, 32], name='weight3', initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([32]))
layer3 = tf.nn.relu(tf.matmul(layer2, W3) + b3)
layer3 = tf.nn.dropout(layer3, keep_prob=keep_prob)
layer3 = tf.contrib.layers.batch_norm(layer3, center=True, scale=True, 
                                          is_training=phase)

W4 = tf.get_variable(shape=[32, 1], name='weight4', initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([1]))
hypothesis = tf.matmul(layer3, W4) + b4
hypothesis = tf.contrib.layers.batch_norm(hypothesis, center=True, scale=True, 
                                          is_training=phase)


cost = tf.reduce_mean(tf.square(hypothesis - Y))
#cost = tf.losses.mean_squared_error(labels=Y, predictions=hypothesis)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(train_size / batch_size)

    for i in range(total_batch):
        batch_xs = X_train[i*batch_size:(i+1)*batch_size]
        batch_ys = y_train[i*batch_size:(i+1)*batch_size].reshape(batch_size,1)
        feed_dict = {X: batch_xs, Y: batch_ys, keep_prob:0.7,phase:True}
        c, _ = sess.run([cost, train], feed_dict=feed_dict)
        avg_cost += c/total_batch
        
    if epoch%10==0:
      print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
      


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


Epoch: 0001 cost = 4109.131759208
Epoch: 0011 cost = 3677.853683036
Epoch: 0021 cost = 3281.173604911
Epoch: 0031 cost = 2915.976688058
Epoch: 0041 cost = 2580.950467355
Epoch: 0051 cost = 2271.782617187
Epoch: 0061 cost = 1989.968840681
Epoch: 0071 cost = 1729.892613002
Epoch: 0081 cost = 1493.839899554
Epoch: 0091 cost = 1279.891434152
Epoch: 0101 cost = 1085.790229143
Epoch: 0111 cost = 911.871135603
Epoch: 0121 cost = 757.511359515
Epoch: 0131 cost = 619.878534807
Epoch: 0141 cost = 500.525866699
Epoch: 0151 cost = 396.459831020
Epoch: 0161 cost = 309.722047642
Epoch: 0171 cost = 236.284427316
Epoch: 0181 cost = 177.176508440
Epoch: 0191 cost = 129.376532636
Epoch: 0201 cost = 91.053934261
Epoch: 0211 cost = 65.698841967
Epoch: 0221 cost = 45.666874204
Epoch: 0231 cost = 34.087409183
Epoch: 0241 cost = 27.662443433
Epoch: 0251 cost = 22.826717022
Epoch: 0261 cost = 20.663586085
Epoch: 0271 cost = 19.998617881
Epoch: 0281 cost = 19.970437023
Epoch: 0291 cost = 19.246843147
Epoch: 03

In [67]:

y_test = y_test.reshape(-1,1)
mse = tf.reduce_mean(tf.square(hypothesis - Y))
print('Accuracy:', sess.run(mse, feed_dict={
      X: X_test, Y: y_test, keep_prob:1, phase:False}))

Accuracy: 46511040000000.0
